[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/python35/IINTS-SDK/blob/main/examples/notebooks/00_Quickstart.ipynb)

# Quickstart

**Goal:** run a clinic-safe preset and export audit + PDF in one pass.

**You’ll learn:**
- how to load a preset
- how to run `run_full`
- where outputs are written


In [ ]:
from __future__ import annotations
from pathlib import Path
import os
import sys
import subprocess


def _find_repo_root() -> Path | None:
    for root in [Path.cwd(), *Path.cwd().parents]:
        if (root / "pyproject.toml").exists() and (root / "src").exists():
            return root
    return None

repo_root = _find_repo_root()
if repo_root is None:
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if not in_colab:
        raise RuntimeError("Run this notebook inside the IINTS-SDK repo or on Colab.")

    if not Path("IINTS-SDK").exists():
        subprocess.check_call(["git", "clone", "https://github.com/python35/IINTS-SDK.git"])
    repo_root = Path("IINTS-SDK").resolve()

os.chdir(repo_root)
sys.path.insert(0, str(repo_root / "src"))
print("Repo root:", repo_root)


## Step 1: Load a preset and algorithm


In [ ]:
import iints
from iints.presets import get_preset
from iints.core.algorithms.fixed_basal_bolus import FixedBasalBolus

preset = get_preset("baseline_t1d")
scenario = dict(preset["scenario"])
scenario["stress_events"] = [e for e in scenario.get("stress_events", []) if e.get("event_type") != "exercise"]

algorithm = FixedBasalBolus(settings={"fixed_basal_rate": 0.4, "carb_ratio": 12.0})


## Step 2: Apply a demo‑safe patient profile


In [ ]:
from iints.validation import load_patient_config_by_name

patient_config = load_patient_config_by_name(preset["patient_config"]).model_dump()
patient_config.update(
    {
        "glucose_decay_rate": 0.01,
        "basal_insulin_rate": 0.4,
        "insulin_sensitivity": 60.0,
        "initial_glucose": 150.0,
    }
)


## Step 3: Run full pipeline (audit + PDF + baseline)


In [ ]:
outputs = iints.run_full(
    algorithm=algorithm,
    scenario=scenario,
    patient_config=patient_config,
    duration_minutes=360,
    time_step=preset["time_step_minutes"],
    seed=42,
    output_dir="results/quickstart",
    enable_profiling=True,
)

outputs.keys()


## Step 4: Inspect outputs


In [ ]:
outputs["results"].head()


In [ ]:
print("Report PDF:", outputs.get("report_pdf"))
print("Audit summary:", outputs.get("audit", {}).get("summary"))


### Recap
You now have a results CSV, an audit trail, a baseline comparison, and a PDF report in `results/quickstart/`.
